In [1]:
# flask
from flask import Flask, render_template, url_for, request
from flask_ngrok import run_with_ngrok

# imports
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import joblib

In [2]:
# preprocessing

# Tokenization des Tags
def delete_element(liste):
    element_to_delete = '>'
    try:
        while True:
            liste.remove(element_to_delete)
    except ValueError:
        pass
    return liste

def Tokenization_Tags(data):
    x = []
    y = []
    for row in data['Tags']:
        x.append(row.split('<'))
    for i in range(len(x)):
        x[i].remove('')
    data['Tags']=x
    return data
stop_words = stopwords.words('english')

def element_to_delete_2(data):
    element_to_remove = '>'
    x = data['Tags']
    y = []
    for i in range(len(x)):
        for j in range(len(x[i])):
            x[i][j]=x[i][j].replace('>','')
    data['Tags'] = x
    return data


# reduction du numbre de tags à 10
def Tags_reducing(data):
    tags = data['Tags']
    unique_tag_word = []
    for row in tags:
        for word in row:
            unique_tag_word.append(word)
    test = pd.DataFrame()
    test['unique_tag_word'] = unique_tag_word
    data_count = test.unique_tag_word.value_counts()
    tags_df = pd.DataFrame(data = np.zeros((len(data),len(data_count[0:10]))),columns=test.unique_tag_word.value_counts()[0:10].sort_values(ascending = False).index)
    a = 0
    while a <4 :
        a = a+1
        for i in range(len(data['Tags'])):
            for word in data['Tags'][i]:
                if word not in tags_df.columns:
                      data['Tags'][i].remove(word)
    empty_list = []
    for i in range(len(data)):
        if data['Tags'][i] == empty_list:
              data = data.drop(i)
    data = data.reset_index()
    for i in range(len(data)):
        for word in data['Tags'][i]:
            tags_df[word][i]=1
    data = data.drop(columns = 'index')
    data = data.reset_index()
    tags_df = tags_df.reset_index()
    data = data.merge(tags_df,on='index',how='left')
    return data

def iml_p5_preprocessing_Tags(data):
    data = Tokenization_Tags(data)
    data = element_to_delete_2(data)
    data = Tags_reducing(data)
    #data = data.drop(columns=['Tags'])
    return data

# tokenization Body
def Tokenization_texte(texte):
    x = []
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    for row in texte:
        x.append(tokenizer.tokenize(row))
    return x

# stemmatization Body
def text_stematization(texte):
    stemmer = SnowballStemmer("english")
    stem_word = []
    stem_sentence = []
    for i in range(len(texte)):
        for word in texte[i]:
            stem_word.append(stemmer.stem(word))
        stem_sentence.append(stem_word)
        stem_word = [] 
    return stem_sentence


# lemmatization Body
def text_lemmatization(texte):
    lemmatizer = WordNetLemmatizer()
    lem_word = []
    lem_sentence = []
    for i in range(len(texte)):
        for word in texte[i]:
            lem_word.append(lemmatizer.lemmatize(word))
        lem_sentence.append(lem_word)
        lem_word = [] 
    return lem_sentence
    
# removing stopwords Body
def delete_stopwords(texte):
    cleaned_sentence = []
    cleaned_texte = []
    for sentence in texte:
        for word in sentence:
            if word not in stopwords.words('english'):
                cleaned_sentence.append(word)
        cleaned_texte.append(cleaned_sentence)
        cleaned_sentence = []
    return cleaned_texte

# removing frequent_words Body
def delete_frequent_words(texte,all_words_df):
    
    frequent_words = list(all_words_Body_df.text_words.value_counts().sort_values(ascending=False)[0:50].index)
    to_keep = ['string', 'http', 'java', 'android', 'list']
    for word in to_keep:
        frequent_words.remove(word)
    cleaned_sentence = []
    cleaned_texte = []
    for sentence in texte:
        for word in sentence:
            if word not in frequent_words:
                cleaned_sentence.append(word)
        cleaned_texte.append(cleaned_sentence)
        cleaned_sentence = []
    return cleaned_texte


def text_reconstruction(texte):
    return " ".join([word for word in texte])  

# text_cleaning Body
def text_cleaning(texte,all_words_df):
    cleaned_texte = Tokenization_texte(texte)
    cleaned_texte = text_stematization(cleaned_texte)
    cleaned_texte = text_lemmatization(cleaned_texte)
    cleaned_texte = delete_stopwords(cleaned_texte)
    cleaned_texte = delete_frequent_words(cleaned_texte,all_words_df)
    reconstructed_text = []
    for i in range(len(cleaned_texte)):
        reconstructed_sentence = text_reconstruction(cleaned_texte[i])
        reconstructed_text.append(reconstructed_sentence)
        reconstructed_sentence = []
    return reconstructed_text

In [6]:
def predict_categorie(predictions):
        cat=['java', 'c#', 'javascript', 'python', 'jquery', 'html', 'php',
       'android', '.net', 'c++', 'sql', 'css', 'mysql', 'string', 'sql-server',
       'c', 'git', 'arrays', 'asp.net', 'linux', 'regex', 'iphone', 'bash',
       'objective-c', 'ios', 'windows', 'ruby', 'eclipse', 'tsql', 'database']
        predicted_cat = []
        empty_list = []
        for i in range((len(cat)-1)):
            if predictions[i][0]==1:
                predicted_cat.append(cat[i])
        if predicted_cat == empty_list:
            predicted_cat.append('unknown')
        return predicted_cat

In [ ]:

all_words_Body_df = pd.read_csv('/home/abdifatah/Ingenieur_Machine_Learning/all_words_Body_df.csv',sep=',')